In [1]:
import json
import pandas as pd
import numpy as np

# Read Json

In [2]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)

with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)

with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)

with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)

with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [3]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [11]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']
in_possession_df = all_events_df[all_events_df['eventName'].isin(['Shot','Pass'])]

In [6]:
# Determine the previous team ID in the same match
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)

# Identify transitions where the team ID changes and it's within the same match
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Create a transitions DataFrame with required columns for merging
transitions['endSec'] = transitions['eventSec'] + 10
transition_times = transitions[['id', 'matchId', 'eventSec', 'endSec']]

<ipython-input-6-6110caba3bd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
<ipython-input-6-6110caba3bd7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['endSec'] = transitions['eventSec'] + 10


In [7]:
# Convert data types to more memory efficient types
transitions['matchId'] = transitions['matchId'].astype('int32')
transitions['eventSec'] = transitions['eventSec'].astype('float32')
transitions['endSec'] = transitions['endSec'].astype('float32')

# Check if converting teamId to a smaller integer type is possible
transitions['teamId'] = pd.to_numeric(transitions['teamId'], downcast='integer')

<ipython-input-7-5f7b6da8d4f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['matchId'] = transitions['matchId'].astype('int32')
<ipython-input-7-5f7b6da8d4f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['eventSec'] = transitions['eventSec'].astype('float32')
<ipython-input-7-5f7b6da8d4f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [21]:
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Add a 10-second window for the transition end time
transitions['transition_end_time'] = transitions['eventSec'] + 10

# Prepare the minimal necessary data for merging
transitions_minimal = transitions[['id', 'matchId', 'eventSec', 'transition_end_time']]
in_possession_df_minimal = in_possession_df[['id', 'matchId', 'eventSec', 'teamId', "subEventName", "positions", "eventName", "playerId"]]

<ipython-input-21-f544830a946a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['transition_end_time'] = transitions['eventSec'] + 10


In [23]:
# Define a function to process chunks
def process_chunk(chunk):
    chunk_merged = chunk.merge(transitions_minimal, on='matchId', suffixes=('', '_trans'))
    return chunk_merged[(chunk_merged['eventSec'] >= chunk_merged['eventSec_trans']) & (chunk_merged['eventSec'] <= chunk_merged['transition_end_time'])]

# Process in chunks
chunk_size = 100000
result_frames = []
for start in range(0, in_possession_df_minimal.shape[0], chunk_size):
    chunk = in_possession_df_minimal.iloc[start:start + chunk_size]
    result_frames.append(process_chunk(chunk))

In [24]:
in_poss_merged_df = pd.concat(result_frames)
in_poss_merged_df = in_poss_merged_df.drop_duplicates(subset='id', keep='first')

In [25]:
in_poss_merged_df

,id,matchId,eventSec,teamId,subEventName,positions,eventName,playerId,id_trans,eventSec_trans,transition_end_time
783,177959175,2499719,10.302366,1609,Simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",Pass,167145,177960133,8.449574,18.449574
957,177959177,2499719,12.548934,1609,Simple pass,"[{'y': 88, 'x': 72}, {'y': 75, 'x': 77}]",Pass,3319,177960133,8.449574,18.449574
1044,177959186,2499719,13.961228,1631,Head pass,"[{'y': 25, 'x': 23}, {'y': 15, 'x': 39}]",Pass,8653,177959186,13.961228,23.961228
1218,177959178,2499719,15.320341,1609,Head pass,"[{'y': 80, 'x': 67}, {'y': 61, 'x': 59}]",Pass,167145,177959186,13.961228,23.961228
1392,177959179,2499719,18.051875,1609,Head pass,"[{'y': 61, 'x': 59}, {'y': 45, 'x': 45}]",Pass,49876,177959186,13.961228,23.961228
...,...,...,...,...,...,...,...,...,...,...,...
939043,253302672,2565927,2813.117759,675,High pass,"[{'y': 27, 'x': 36}, {'y': 90, 'x': 33}]",Pass,14723,253302669,2808.463744,2818.463744
939197,253302675,2565927,2817.686043,675,Simple pass,"[{'y': 90, 'x': 33}, {'y': 94, 'x': 45}]",Pass,4501,253302669,2808.463744,2818.463744
940276,253302650,2565927,2874.769542,682,Simple pass,"[{'y': 16, 'x': 7}, {'y': 6, 'x': 27}]",Pass,3695,253302650,2874.769542,2884.769542
940430,253302651,2565927,2876.959208,682,Simple pass,"[{'y': 6, 'x': 27}, {'y': 4, 'x': 40}]",Pass,286994,253302650,2874.769542,2884.769542


# in_poss_merged_df is ready (update below to create final_df)

# Feature engineering - Pass DF - Pass Motifs

In [26]:
passes_df = in_poss_merged_df

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

In [27]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [28]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [29]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [30]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [31]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [32]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.047619,0.085714,0.085714,0.780952
10531---2499736,0.029787,0.080851,0.080851,0.808511
10531---2499741,0.034043,0.072340,0.072340,0.821277
10531---2499757,0.021583,0.082734,0.079137,0.816547
10531---2499766,0.030675,0.104294,0.110429,0.754601
...,...,...,...,...
756---2565880,0.050388,0.089147,0.089147,0.771318
756---2565892,0.048246,0.109649,0.114035,0.728070
756---2565904,0.032922,0.053498,0.053498,0.860082


In [33]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [34]:
motif_stats

,mean,std
Motif,,
ABAB,0.035186,0.014412
ABCA,0.082267,0.016402
ABCB,0.082798,0.016277
ABCD,0.799748,0.036292


In [35]:
motif_ratios.to_csv('transition_motif_ratios_df.csv', index=True)

# Feature engineering - Shot DF

In [44]:
shot_df = in_poss_merged_df[in_poss_merged_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-44-850d7acc142f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [45]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-45-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [46]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-46-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [47]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [55]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499726,2,0.000,0.000,1.00
10531---2499736,2,0.500,0.500,0.00
10531---2499741,1,0.000,0.000,1.00
10531---2499757,1,0.000,0.000,1.00
10531---2499766,1,0.000,0.000,1.00
...,...,...,...,...
756---2565880,6,0.000,0.500,0.50
756---2565892,4,0.500,0.500,0.00
756---2565904,5,0.000,0.800,0.20


In [ ]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [56]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499726,2,0.000,0.000,1.00
10531---2499736,2,0.500,0.500,0.00
10531---2499741,1,0.000,0.000,1.00
10531---2499757,1,0.000,0.000,1.00
10531---2499766,1,0.000,0.000,1.00
...,...,...,...,...
756---2565880,6,0.000,0.500,0.50
756---2565892,4,0.500,0.500,0.00
756---2565904,5,0.000,0.800,0.20


# Feature engineering - Pass DF - Pass Types

In [57]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x / 33.333) + 1
        if zone == 4:
            zone = 3
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

In [58]:
passes_df

,id,matchId,eventSec,teamId,subEventName,positions,eventName,playerId,id_trans,eventSec_trans,transition_end_time,uniqueTeamId,Zone
783,177959175,2499719,10.302366,1609,Simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",Pass,167145,177960133,8.449574,18.449574,1609---2499719,2
957,177959177,2499719,12.548934,1609,Simple pass,"[{'y': 88, 'x': 72}, {'y': 75, 'x': 77}]",Pass,3319,177960133,8.449574,18.449574,1609---2499719,3
1044,177959186,2499719,13.961228,1631,Head pass,"[{'y': 25, 'x': 23}, {'y': 15, 'x': 39}]",Pass,8653,177959186,13.961228,23.961228,1631---2499719,1
1218,177959178,2499719,15.320341,1609,Head pass,"[{'y': 80, 'x': 67}, {'y': 61, 'x': 59}]",Pass,167145,177959186,13.961228,23.961228,1609---2499719,3
1392,177959179,2499719,18.051875,1609,Head pass,"[{'y': 61, 'x': 59}, {'y': 45, 'x': 45}]",Pass,49876,177959186,13.961228,23.961228,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
939043,253302672,2565927,2813.117759,675,High pass,"[{'y': 27, 'x': 36}, {'y': 90, 'x': 33}]",Pass,14723,253302669,2808.463744,2818.463744,675---2565927,2
939197,253302675,2565927,2817.686043,675,Simple pass,"[{'y': 90, 'x': 33}, {'y': 94, 'x': 45}]",Pass,4501,253302669,2808.463744,2818.463744,675---2565927,1
940276,253302650,2565927,2874.769542,682,Simple pass,"[{'y': 16, 'x': 7}, {'y': 6, 'x': 27}]",Pass,3695,253302650,2874.769542,2884.769542,682---2565927,1
940430,253302651,2565927,2876.959208,682,Simple pass,"[{'y': 6, 'x': 27}, {'y': 4, 'x': 40}]",Pass,286994,253302650,2874.769542,2884.769542,682---2565927,1


In [59]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [60]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [61]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [62]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

In [ ]:
passes_df

In [ ]:
np.unique(passes_df.subEventName)

In [ ]:
passes_df

In [ ]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
"""
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Analyzing pass length
q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)

# Grouping, Counting, and Pivoting
grouped = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()
pivot_table = grouped.unstack(fill_value=0)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()

# Final DataFrame
final_df = pd.concat([pivot_table, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

final_df.head()  # Display the top rows of the DataFrame"""


In [63]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

q1, q2, q3 = passes_df['calculated_pass_length'].quantile([0.33, 0.66, 1.0])
passes_df['pass_length_type'] = pd.cut(passes_df['calculated_pass_length'], bins=[0, q1, q2, q3], labels=['Short', 'Middle', 'Long'], include_lowest=True)
pass_length_stats = passes_df.groupby(['uniqueTeamId', 'pass_length_type']).size().unstack(fill_value=0)
average_pass_length = passes_df.groupby('uniqueTeamId')['calculated_pass_length'].mean()


# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count, pass_length_stats, average_pass_length.rename('average_pass_length')], axis=1)

In [64]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass,Short,Middle,Long,average_pass_length
uniqueTeamId,,,,,,,,,,,,,,,,
10531---2499726,5,21,5,42,34,5,40,44,17,31,81,101,57,69,87,25.923929
10531---2499736,9,15,9,48,30,9,48,45,25,33,87,118,89,76,73,25.496242
10531---2499741,4,19,5,47,43,6,38,68,8,28,96,114,69,76,93,26.409845
10531---2499757,7,17,17,54,39,12,61,49,25,41,105,135,85,87,109,26.078901
10531---2499766,6,15,4,42,34,2,28,27,8,25,78,63,35,43,88,30.241780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,7,15,14,33,42,11,43,59,37,36,86,139,80,75,106,28.875240
756---2565892,10,20,7,37,33,8,33,53,30,37,78,116,74,78,79,25.158064
756---2565904,12,19,8,54,28,4,64,36,21,39,86,121,80,64,102,27.640545


## Pass Types

In [73]:
high_pass_types = ['High pass', 'Cross', 'Launch']
# Define low pass types based on exclusion
all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# Group by 'uniqueTeamId' and count passes
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# Calculate high pass and low pass totals
pass_counts['High_pass_Total'] = pass_counts[high_pass_types].sum(axis=1)
pass_counts['Low_pass_Total'] = pass_counts[low_pass_types].sum(axis=1)

# Calculate overall high pass and low pass ratios
pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total'] = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Smart_pass_Ratio_Total'] = pass_counts['Smart pass'] / pass_counts['Total_Passes']


# Now, we'll prepare the final DataFrame with the required columns
final_columns = ['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total', 'Smart_pass_Ratio_Total']
final_pass_types_df = pass_counts[final_columns]

# If needed, calculate zone-specific ratios (demonstrated for High pass, replicate for Low pass as necessary)
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_specific_passes = passes_df[passes_df['Zone'] == zone]
    zone_pass_counts = zone_specific_passes.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)
    
    # High pass total and ratio for the zone
    zone_pass_counts['High_pass_Total_Zone'] = zone_pass_counts[high_pass_types].sum(axis=1)
    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
    
    # Low pass total and ratio for the zone
    zone_pass_counts['Low_pass_Total_Zone'] = zone_pass_counts[low_pass_types].sum(axis=1)
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']

<ipython-input-73-baa8d20fec00>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
<ipython-input-73-baa8d20fec00>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']


In [74]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,Smart_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1
uniqueTeamId,,,,,,,,,,
10531---2499726,213,0.145540,0.849765,0.004695,0.060606,0.939394,0.185185,0.814815,0.229885,0.758621
10531---2499736,238,0.155462,0.836134,0.008403,0.077778,0.911111,0.209302,0.767442,0.200000,0.800000
10531---2499741,238,0.168067,0.823529,0.008403,0.153846,0.830769,0.105263,0.894737,0.202247,0.797753
10531---2499757,281,0.153025,0.832740,0.014235,0.095238,0.876190,0.203704,0.777778,0.180328,0.819672
10531---2499766,166,0.234940,0.753012,0.012048,0.144737,0.828947,0.214286,0.785714,0.328947,0.671053
...,...,...,...,...,...,...,...,...,...,...
756---2565880,261,0.187739,0.800766,0.011494,0.112069,0.879310,0.322581,0.645161,0.192771,0.807229
756---2565892,231,0.155844,0.835498,0.008658,0.103774,0.886792,0.288889,0.688889,0.150000,0.850000
756---2565904,246,0.231707,0.756098,0.012195,0.156627,0.819277,0.333333,0.636364,0.253846,0.746154


# Pass_Connectivity

In [ ]:
pd.DataFrame(passes_df.groupby("matchId"))

In [ ]:
passes_df

In [65]:
from collections import defaultdict
import networkx as nx

In [66]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [68]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [104]:
conn_dict = {**{key: [value] for d in connectivity_results for key, value in d.items()}}
connectivity_df = pd.DataFrame(conn_dict)
connectivity_df = connectivity_df.T
connectivity_df.rename(columns = {0:'Connectivity'}, inplace = True) 

In [105]:
connectivity_df.to_csv('transition_connectivity_df.csv', index=True)

# Merge DataFrames

In [108]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side', "Short", "Middle", "Long"]):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [109]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,Smart_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,...,Side Pass Ratio,Short Ratio,Middle Ratio,Long Ratio,average_pass_length,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio Ratio,Near Shots Ratio,Connectivity
10531---2499726,0.047619,0.085714,0.085714,0.780952,213,0.145540,0.849765,0.004695,0.060606,0.939394,...,0.474178,0.267606,0.323944,0.408451,25.923929,2.0,0.000,0.000000,1.00,5.051282
10531---2499736,0.029787,0.080851,0.080851,0.808511,238,0.155462,0.836134,0.008403,0.077778,0.911111,...,0.495798,0.373950,0.319328,0.306723,25.496242,2.0,0.500,0.002101,0.00,5.461538
10531---2499741,0.034043,0.072340,0.072340,0.821277,238,0.168067,0.823529,0.008403,0.153846,0.830769,...,0.478992,0.289916,0.319328,0.390756,26.409845,1.0,0.000,0.000000,1.00,5.576923
10531---2499757,0.021583,0.082734,0.079137,0.816547,281,0.153025,0.832740,0.014235,0.095238,0.876190,...,0.480427,0.302491,0.309609,0.387900,26.078901,1.0,0.000,0.000000,1.00,5.589744
10531---2499766,0.030675,0.104294,0.110429,0.754601,166,0.234940,0.753012,0.012048,0.144737,0.828947,...,0.379518,0.210843,0.259036,0.530120,30.241780,1.0,0.000,0.000000,1.00,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.050388,0.089147,0.089147,0.771318,261,0.187739,0.800766,0.011494,0.112069,0.879310,...,0.532567,0.306513,0.287356,0.406130,28.875240,6.0,0.000,0.001916,0.50,5.021978
756---2565892,0.048246,0.109649,0.114035,0.728070,231,0.155844,0.835498,0.008658,0.103774,0.886792,...,0.502165,0.320346,0.337662,0.341991,25.158064,4.0,0.500,0.002165,0.00,6.666667
756---2565904,0.032922,0.053498,0.053498,0.860082,246,0.231707,0.756098,0.012195,0.156627,0.819277,...,0.491870,0.325203,0.260163,0.414634,27.640545,5.0,0.000,0.003252,0.20,6.653846
756---2565908,0.049669,0.089404,0.096026,0.764901,305,0.144262,0.839344,0.016393,0.104575,0.869281,...,0.445902,0.308197,0.337705,0.354098,26.000048,5.0,0.400,0.001311,0.20,5.582418


In [110]:
final_df.to_csv('final_transition_in_poss_df.csv', index=True)

# END OF FILE